# Package Installation
At the console, within the `cs207project` directory, you can use
```python 
pip install -e .
```
or equivalently
```python
python setup.py install
```
At this point, four subpackages will be available to you:
1. `timeseries`
2. `TimeseriesDB`
3. `Similarity`
4. `cs207rbtree`

In [1]:
import timeseries, TimeseriesDB, Similarity
import cs207rbtree.RedBlackTree as Database

['/Users/sarahanoke/GitHub/cs207project/TimeseriesDB', '/Users/sarahanoke/GitHub/cs207project', '/Users/sarahanoke/GitHub/cs207project/timeseries/Similarity/VantagePointDatabases', '/Users/sarahanoke/GitHub', '/Users/sarahanoke/GitHub/cs207project/TimeseriesDB', '/Users/sarahanoke/GitHub/cs207project', '/Users/sarahanoke/GitHub/cs207project/timeseries', '/Users/sarahanoke/GitHub/cs207project', '', '/Users/sarahanoke/GitHub/cs207project', '/Users/sarahanoke/anaconda/lib/python35.zip', '/Users/sarahanoke/anaconda/lib/python3.5', '/Users/sarahanoke/anaconda/lib/python3.5/plat-darwin', '/Users/sarahanoke/anaconda/lib/python3.5/lib-dynload', '/Users/sarahanoke/anaconda/lib/python3.5/site-packages/Sphinx-1.4.1-py3.5.egg', '/Users/sarahanoke/anaconda/lib/python3.5/site-packages/setuptools-23.0.0-py3.5.egg', '/Users/sarahanoke/anaconda/lib/python3.5/site-packages', '/Users/sarahanoke/anaconda/lib/python3.5/site-packages/aeosa', '/Users/sarahanoke/anaconda/lib/python3.5/site-packages/IPython/ex

In [2]:
dir(Database)

['Color',
 'DBDB',
 'RedBlackNode',
 'RedBlackNodeRef',
 'RedBlackTree',
 'Storage',
 'ValueRef',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'connect',
 'os',
 'pickle',
 'portalocker',
 'struct']

# Using the Red-Black Tree

Below is a function that will allow us to visualize our tree (copied from CS207 Fall 2016 [lecture notes](https://github.com/iacs-cs207/cs207-2016/blob/master/lectures/BinarySearchTrees.ipynb)). 

Next, we create a tree, add a few nodes to it, and retrieve their contents.

In [3]:
demoDB = Database.connect("/tmp/test1.dbdb")

In [4]:
demoDB.set("rahul",    81)
demoDB.set("pavlos",   20)
demoDB.set("sarah",    29)
demoDB.set("courtney", 11)
demoDB.set("andrew",   12)
demoDB.set("laura",    81)

In [5]:
demoDB.get("sarah")

29

In [6]:
demoDB.get("laura")

81